# 2024-2 언어데이터과학 19강 (2024-11-28) 실습 (1) `gensim` 패키지와 Word2Vec 모델 훈련

## 오늘의 목표

1. 모두의 말뭉치 '온라인 대화 말뭉치'의 발화 자료로 Word2Vec Skip-gram 모델을 훈련시킬 수 있다.
2. 단어 사이의 의미 유사성을 벡터들의 코사인 유사도로 계량화하여 설명할 수 있다.
3. t-SNE 기법을 사용하여 단어 벡터들을 2차원 평면에 시각화할 수 있다.
4. Word2Vec 모델을 사용하여 연령대에 따른 단어의 분포 차이를 설명할 수 있다.

## 0. 준비

### 파이썬 모듈 설치하기

Python에서 Word2Vec 모델을 사용하기 위해서는 `gensim` 모듈이 필요하다.

In [1]:
%pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 20.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 7.4 MB/s eta 0:00:000:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.1
    Uninstalling numpy-2.1.1:
      Successfully uninstalled numpy-2.1.1
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
Note: you may need to restart the kernel to use updated packages.


## 1. 데이터 가공

12강에서 만든 모두의 말뭉치 [온라인 대화 말뭉치] 파일을 읽고 `gensim` 모듈에서 사용 가능한 코퍼스로 가공하자.

In [1]:
import pandas as pd # data frame
from gensim.models import Word2Vec # word embedding
from tqdm import tqdm # progress bar

### 데이터 파일 읽기

In [2]:
DATA_PATH = '../data/nikl/NIKL_OM_form_age_sex.csv.tar.gz'

In [3]:
utterances = pd.read_csv(DATA_PATH, compression='gzip', on_bad_lines='skip')
utterances

,data/nikl/NIKL_OM_form_age_sex.csv,age,sex
0,안녕하세요,20대,여성
1,️,20대,여성
2,이거 해봐요><,20대,여성
3,"나의 직장인 멘탈 성향은 [안챙겨도 잘커요, 탕비실 선인장] 당신의 멘탈 성향은 ...",20대,여성
4,아앗...,20대,여성
...,...,...,...
2977836,아잌ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,20대,여성
2977837,가즈아,20대,여성
2977838,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ가즈아!,20대,여성
2977839,달려달려,20대,여성


In [4]:
utterances = pd.read_csv(DATA_PATH, compression='gzip', on_bad_lines='skip')
utterances.dropna(inplace=True)
utterances.rename(columns={utterances.columns[0]: 'form'}, inplace=True)
utterances

,form,age,sex
0,안녕하세요,20대,여성
1,️,20대,여성
2,이거 해봐요><,20대,여성
3,"나의 직장인 멘탈 성향은 [안챙겨도 잘커요, 탕비실 선인장] 당신의 멘탈 성향은 ...",20대,여성
4,아앗...,20대,여성
...,...,...,...
2977835,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,20대,여성
2977836,아잌ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,20대,여성
2977837,가즈아,20대,여성
2977838,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ가즈아!,20대,여성


### 데이터 형변환

일반적으로 Python의 여러 라이브러리에서 코퍼스를 다룰 때는 한 문장을 단어들의 리스트로 표현하고, 코퍼스 전체를 문장들의 리스트로 표현한다.

지금 가지고 있는 데이터프레임에서는 문장에 해당하는 발화가 `str` 자료형이므로, `str.split()` 메소드를 사용하여 단어들의 리스트로 만들어 주자.

In [5]:
corpus = utterances['form'].apply(str.split) # edit this line
print(corpus[:5])

0                                              [안녕하세요]
1                                                  [️]
2                                          [이거, 해봐요><]
3    [나의, 직장인, 멘탈, 성향은, [안챙겨도, 잘커요,, 탕비실, 선인장], 당신의...
4                                              [아앗...]
Name: form, dtype: object


## 2. Word2Vec 모델 훈련

### 모델 초기화

In [6]:
model = Word2Vec(
    sg=1,
    min_count=5,
    vector_size=100,
    window=2,
    negative=5,
)

### Vocabulary 구축

In [7]:
model.build_vocab(corpus_iterable=tqdm(corpus))

100%|██████████| 2977840/2977840 [00:04<00:00, 665081.27it/s]


### 모델 훈련시키기

In [8]:
model.train(
    corpus_iterable=corpus,
    total_examples=model.corpus_count,
    epochs=5
)
# (trained_word_count, raw_word_count)

(32845001, 46433495)

### 훈련 결과

Vocabulary 확인

In [9]:
# 빈도순 상위 30개 단어
print(model.wv.index_to_key[:30])

['아', 'ㅋㅋ', 'ㅋㅋㅋ', '저는', '진짜', '근데', '너무', 'ㅎㅎ', '다', '저도', '좀', 'ㅋㅋㅋㅋ', '잘', '그', '나', '네', '더', '많이', '전', '그냥', '오', '난', 'ㅠㅠ', '맞아요', 'ㅋㅋㅋㅋㅋ', '안', '오늘', '저', '그래서', '그럼']


단어 벡터 확인

In [10]:
print(model.wv['ㅋㅋ'])

[-1.96811646e-01 -4.67672229e-01 -1.10853052e+00  4.63435471e-01
 -5.67317545e-01 -6.04940485e-03 -7.45956749e-02  3.10505629e-01
 -4.31377999e-02  3.81155968e-01 -3.01036030e-01  2.14202419e-01
 -5.41801274e-01  1.21644221e-01 -6.11828983e-01 -2.83032507e-01
  4.67197955e-01  2.89971739e-01  5.34604132e-01 -2.05180943e-01
  1.70622602e-01 -1.80923313e-01  4.77950066e-01 -5.78660965e-01
  1.00205755e+00 -2.76321352e-01  3.92449677e-01  4.92483340e-02
 -5.47579788e-02  6.62056684e-01  4.57391255e-02 -1.40982270e-01
  3.52476746e-01 -4.21473205e-01 -2.55408496e-01  1.45491183e-01
  2.56319135e-01 -2.54204739e-02  1.60508513e-01 -7.58712232e-01
  1.95780039e-01 -3.67543161e-01  2.12812692e-01 -5.20948291e-01
  2.05238612e-04 -2.80146331e-01 -4.48927656e-02 -1.20049846e+00
  6.86576784e-01  6.39382482e-01 -8.51660967e-02 -4.70205426e-01
 -3.36996108e-01  3.82322706e-02 -1.25026479e-01  4.50485170e-01
  1.93788469e-01  9.75721180e-01 -1.72251418e-01  1.30938619e-01
 -2.12128623e-03 -2.43914

## 3. Word2Vec 모델 활용

### `gensim`의 주요 기능



코사인 유사도 계산

In [11]:
print(model.wv.similarity(w1='펜', w2='볼펜'))
print(model.wv.similarity(w1='펜', w2='맥주'))

0.91619056
0.60392934


평행사변형 모형

In [12]:
# 아빠 : 엄마 = X : 할머니
# 아빠 - 엄마 = X - 할머니
# X = 할머니 + 아빠 - 엄마
print(model.wv.most_similar(positive=['할머니', '아빠'], negative=['엄마'])) # EDIT THIS LINE

[('할아버지', 0.8138434290885925), ('name11', 0.8039007186889648), ('너희', 0.8033213019371033), ('name10', 0.8007969856262207), ('니네', 0.7998496890068054), ('name7이', 0.7930594086647034), ('이모가', 0.7900922894477844), ('name12', 0.789970874786377), ('아빠랑', 0.7899647951126099), ('동생', 0.7849502563476562)]


우리 중에 스파이가 있는 것 같아

In [13]:
print(model.wv.doesnt_match(['ㅋㅋ', 'ㅎㅎ', '^^', 'ㅠㅠ']))

ㅠㅠ


### '완전'과 '아주'의 이웃 비교

In [14]:
model.wv.most_similar(['완전'])

[('짱', 0.7012233138084412),
 ('되게', 0.643863320350647),
 ('디게', 0.6269715428352356),
 ('대박', 0.6215198636054993),
 ('무지', 0.6200389266014099),
 ('넘나', 0.6100852489471436),
 ('장난', 0.6009043455123901),
 ('왕', 0.5968322157859802),
 ('너뮤', 0.5880658626556396),
 ('진쨔', 0.5750538110733032)]

In [15]:
model.wv.most_similar(['아주'])

[('매우', 0.7714052796363831),
 ('굉장히', 0.6949530243873596),
 ('상당히', 0.6894147396087646),
 ('무지', 0.6877973675727844),
 ('무척', 0.6787397861480713),
 ('넘나', 0.6555966734886169),
 ('그저', 0.6456524133682251),
 ('운이', 0.6256696581840515),
 ('훨씬', 0.6198590993881226),
 ('되게', 0.6186764240264893)]

### 모델 저장하기

In [ ]:
%mkdir -p ../models
model.save('../models/word2vec-modu-online')

: 